In [50]:
#-- Importing modules
import flopy
import numpy as np
import pandas as pd
import flopy.modflow as fpm

In [51]:
#--importing data file
import csv

with open('coord.txt', newline='') as csvfile:
    data = list(csv.reader(csvfile))

print(data)

[['x', 'y'], [' 1', '1537.3919', '6482.2855'], [' 2', '1619.4925', '5350.2436'], [' 3', '1189.2087', '4426.7064'], [' 4', '1610.6249', '3517.6656'], [' 5', '95.2641', '2598.2555'], [' 6', '1800.4278', '2359.5935'], [' 7', '78.0893', '1727.8779'], ['8', '1708.18', '1586.7347'], [' 9', '115.4608', '581.3898'], [' 10', '1348.1983', '475.2261'], [' 11', '1909.8702', '-185.123'], [' 12', '1627.0996', '6571.9932'], [' 13', '1709.2002', '5439.9513'], [' 14', '1437.9059', '564.9337'], [' 15', '1537.3919', '6571.9932'], []]


In [59]:
#-- Defining boundries for DIS package
Lx = 5000
Ly = 10000
ztop = 0
zbot = -30
nlay = 2
nrow = 100
ncol = 100
delr = Lx / ncol
delc = Ly / nrow
delv = (ztop - zbot) / nlay
perlen = 20

In [60]:
#--Generating model name
model_name = 'Draft_SP'
mf = flopy.modflow.Modflow(model_name, exe_name='mf2005')

In [61]:
disc = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, delc=delc,
                               top=ztop, botm = zbot)

In [62]:
#--Output Control
spd = {(0, 0): ['print head', 'print budget', 'save head', 'save budget']}
oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd, compact=True)

In [63]:
#--BCF File
laycon = 0 #confined
tran = 1.0 
bcf = flopy.modflow.mfbcf.ModflowBcf(mf, laycon = 0, tran = 1)

In [64]:
#--Defining Head Boundary Conditions
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
ibound[:, :, 0] = -1
ibound[:, :, -1] = -1
strt = np.ones((nlay, nrow, ncol), dtype=np.float32)
strt[:, :, 0] = 10.
strt[:, :, -1] = 10.
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

In [65]:
#-- PCG File
pcg = flopy.modflow.mfpcg.ModflowPcg(mf, mxiter = 20, iter1 = 30)

In [66]:
#-- Defining Recharge and Well Package
fpm.ModflowRch(mf, rech=0.001)
Q = { 0: [[0, 1537.3919, 6482.2855, 0.5], [0, 1619.4925, 5350.2436, 0.5], [0, 1189.2087, 4426.7064, 0.5], [0, 1610.6249, 3517.6656, 0.5],
         [0, 95.2641, 2598.2555, 0.5],[0, 1800.4278, 2359.5935, 0.5], [0, 78.0893, 1727.8779, 0.5], [0, 1708.18, 1586.7347, 0.5], [0, 115.4608, 581.3898, 0.5],
         [0, 1348.1983, 475.2261, 0.5], [0, 1909.8702, -185.123, 0.5], [0, 1627.0996, 6571.9932, 0.5], [0, 1709.2002, 5439.9513, 0.5],
         [0, 1437.9059, 564.9337, 0.5], [0, 1537.3919, 6571.9932, 0.5]]} #-- [layer, row, column, injection rate]
fpm.ModflowWel(mf, stress_period_data = Q)


    MODFLOW Well Package Class.

    Parameters
    ----------
    model : model object
        The model object (of type :class:`flopy.modflow.mf.Modflow`) to which
        this package will be added.
    ipakcb : int
        A flag that is used to determine if cell-by-cell budget data should be
        saved. If ipakcb is non-zero cell-by-cell budget data will be saved.
        (default is 0).
    stress_period_data : list of boundaries, or recarray of boundaries, or
        dictionary of boundaries
        Each well is defined through definition of
        layer (int), row (int), column (int), flux (float).
        The simplest form is a dictionary with a lists of boundaries for each
        stress period, where each list of boundaries itself is a list of
        boundaries. Indices of the dictionary are the numbers of the stress
        period. This gives the form of:

            stress_period_data =
            {0: [
                [lay, row, col, flux],
                [lay, r

In [67]:
#-- Linkag with mt3d
lmt = flopy.modflow.ModflowLmt(mf, output_file_name = 'mt3d_link.ftl')

In [68]:
#--Writing Input Files
mf.write_input()
#--And to run the model
mf.run_model()

FloPy is using the following  executable to run the model: C:\WRDAPP\mf2005.1\bin\mf2005.EXE

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: Draft_SP.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2020/08/18 19:33:21



(False, [])